# SuperMapRealigner - Demo
## Import

In [1]:
import os, subprocess, glob, tqdm, json
import numpy as np
import cv2

from utils.segment import *
from utils.match import *

## Global variables

In [2]:
SEG_MAPS_FOLDER = ('data', 'demo', 'segmented')
CONTROL_PTS_FOLDER = ('data', 'demo', 'control_points')
PREPROC_MAPS_FOLDER = ('workshop', 'skeletonized')
SG_OUTPUT_FOLDER = ('workshop', 'sg_output')
REALIGN_OUTPUT_FOLDER = ('output', 'demo')
ANCHOR_FOLDER = ('data', 'demo')

PARAMS = {
    'corpus': {
        'city_name': 'Paris',
    },
    'anchor': {
        'admin_level': 8, # cf. https://wiki.openstreetmap.org/wiki/Key:admin_level
        'streetwidth_coef': 4, # affect the width of the roads in the reference anchor map
        'image_maxsize': 15000,
    },
    'realign': {
        'resize': 1400,
        'nms_radius': 5,
        'max_keypoints': -1,
        'ransac_radius': 50,
    },
}

# Create necessary directories
os.makedirs(os.path.join(*PREPROC_MAPS_FOLDER), exist_ok=True)
os.makedirs(os.path.join(*REALIGN_OUTPUT_FOLDER), exist_ok=True)

## Main
### Create reference anchor (if necessary)
The anchor map is created by querying OpenStreetMap via the Overpass API. It is based on the city_name you provide in the PARAMS. PS: You might want to tweak the request to the Overpass API if you are trying to realign maps covering larger areas.

In [3]:
createAnchorMap(PARAMS, ANCHOR_FOLDER)

C:\Users\Rémi Petitpierre\Desktop\SuperMapRealigner\utils\match.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['width'].loc[df['highway_type'] == type_] = 1 + highway_types[type_]


### Preprocess segmented maps

In [10]:
# Preprocess the reference anchor map
anchor_name, anchor_scale, anchor_shape = preprocess_segmented_maps(os.path.join(*ANCHOR_FOLDER, "anchor.png"),
                                                                    PREPROC_MAPS_FOLDER, PARAMS['realign']['resize'])
anchor_scale =  max(anchor_shape[:2]) / PARAMS['realign']['resize']

# Preprocess and store scale factors for target segmented maps
scale_factor, orig_shape = dict(), dict()
input_pairs = ''
segmented_maps_paths = sorted(glob.glob(os.path.join(*SEG_MAPS_FOLDER, '*.png')))
for image_path in tqdm.tqdm(segmented_maps_paths):
    
    image_name, image_scale, image_shape = preprocess_segmented_maps(image_path, PREPROC_MAPS_FOLDER,
                                                                     PARAMS['realign']['resize'])
    
    scale_factor[image_name.split('.')[0]] = image_scale
    orig_shape[image_name.split('.')[0]] = image_shape[:2]
    
    # Add target-reference pair
    input_pairs += f"{image_name} {anchor_name}\n"
    
# Save input pairs (target-reference) to a file
with open(os.path.join(*PREPROC_MAPS_FOLDER, 'input_pairs.txt'), 'w+') as f:
    f.write(input_pairs[:-1])

100%|██████████████████████████████████████████████████████████████████████████████████| 49/49 [00:33<00:00,  1.46it/s]


### Compute and pre-match Local Features

In [11]:
cmd = f"python external/SuperGluePretrainedNetwork/match_pairs.py --superglue outdoor \
--resize {PARAMS['realign']['resize']} \
--max_keypoints {PARAMS['realign']['max_keypoints']} \
--nms_radius {PARAMS['realign']['nms_radius']} \
--input_dir {os.path.join(*PREPROC_MAPS_FOLDER)} \
--input_pairs {os.path.join(*PREPROC_MAPS_FOLDER, 'input_pairs.txt')} \
--output_dir {os.path.join(*SG_OUTPUT_FOLDER)}"

# Apply pretrained SuperGlue model on input pairs
subprocess.call(cmd, shell=True)

0

### Realigner
#### Demo on validation data

In [12]:
output = realign(SG_OUTPUT_FOLDER, ANCHOR_FOLDER, anchor_name, anchor_scale, 
                 scale_factor, orig_shape, PARAMS['realign']['ransac_radius'],
                 with_validation=True, control_pts_folder=CONTROL_PTS_FOLDER)

# Compute and display success rate and median residual error
total, residuals = 0, []
for o in output:
    if o['success']:
        total += 1
        residuals += o['residuals']

print(f'Percentage correctly realigned: {"{:.2f}".format(100*total/len(output))} %.')
print(f'Median residual error: {"{:.2f}".format(np.median(residuals))} px.')

100%|██████████████████████████████████████████████████████████████████████████████████| 49/49 [01:42<00:00,  2.08s/it]

Percentage correctly realigned: 75.51 %.
Median residual error: 16.69 px.


#### Inference on any segmented data

In [16]:
output = realign(SG_OUTPUT_FOLDER, ANCHOR_FOLDER, anchor_name, anchor_scale, 
                 scale_factor, orig_shape, PARAMS['realign']['ransac_radius'])

100%|██████████████████████████████████████████████████████████████████████████████████| 49/49 [01:42<00:00,  2.08s/it]


### Export

In [15]:
with open(os.path.join(*REALIGN_OUTPUT_FOLDER, "output.json"), "w") as export_file:
    json.dump(output, export_file)